In [48]:
import pandas as pd
import numpy as np

import torch
torch.cuda.is_available()

False

In [17]:
df = pd.read_excel("/media/cattiaux/DATA/Wassati/team_data/lp3/Umfrage ZSP 2022 - Coop Gruppe(1-3749).xlsx", engine='openpyxl')
# Rename columns for readability
df = df.rename(columns={
    'Was zeichnet nach Ihren Erfahrungen einen guten Vorgesetzten / eine gute Vorgesetzte aus und was erwarten Sie von ihm respektive ihr? (Stichworte oder Fliesstext)': 'Comments'})
# df['year'] = pd.DatetimeIndex(df["Startzeit"]).year
# Replace 'Deutsch' with 'de'
df['Language'] = df['Language'].replace('Deutsch', 'de')
# Replace 'Français (France)' with 'fr'
df['Language'] = df['Language'].replace('Français (France)', 'fr')
# Replace 'English (United Kingdom)' with 'en'
df['Language'] = df['Language'].replace('English (United Kingdom)‎', 'en')
# Replace 'Italiano' with 'it'
df['Language'] = df['Language'].replace('Italiano', 'it')
# df.to_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments.csv", header=True, index=False)

df2 = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/hlt_open_answer_with_language.csv")
df2 = df2.rename(columns={'languages':'Language', 'answer':"Comments"})

df_merge = df.append(df2, ignore_index=True)
# df_merge.to_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge.csv", header=True, index=False)

In [3]:
df_merge = pd.read_csv("/media/cattiaux/DATA/Wassati/team_data/lp3/lp3_comments_merge.csv")
# Filter rows where the 'Language' column is in the specified values
desired_languages = ['en', 'it', 'fr', 'de']
df_merge = df_merge[df_merge['Language'].isin(desired_languages)]

# Replace newline characters with a space
df_merge['Comments'] = df_merge['Comments'].str.replace('\n', ' ')

df_merge = df_merge.dropna(subset=['Comments'])

# Mapping of language codes to full language names
language_mapping = {
    'fr': 'French',
    'de': 'German',
    'es': 'Spanish',
    'it': 'Italian'
}

# Add the prefix based on the language value
df_merge['comment_to_translate'] = df_merge.apply(lambda row: f"Translate {language_mapping.get(row['Language'], row['Language'])} to English: {row['Comments']}" if row['Language'] != 'en' else row['Comments'], axis=1)


In [4]:
df_merge.Language.value_counts()

de    5516
fr    1411
it     234
en     200
Name: Language, dtype: int64

In [ ]:
3. Créer la liste des stopwords
(
4. Créer la liste des ngrams (bigram - trigram)
5. Créer son custom vocabulaire
)
6. Lancer un premier Bertopic et regarder ce que ça donne (cb de topic, bien réparti ou pas, clair ou pas)
(
7bis. Traduire l'ensemble en anglais
)

In [6]:
# Use a pipeline as a high-level helper
from transformers import pipeline

# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create translation pipelines for each language pair
german_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-de-en", max_length=1350, device=device)
french_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-fr-en", max_length=1350, device=device)
italian_to_english = pipeline("translation", model="Helsinki-NLP/opus-mt-it-en", max_length=1350, device=device)

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [47]:
def translate_comment(row):
    comment = row['Comments']
    lang = row['Language']
    
    if lang == 'fr':
        # Translate French to English
        return french_to_english(comment)[0]['translation_text']
    elif lang == 'de':
        # Translate German to English
        return german_to_english(comment)[0]['translation_text']
    elif lang == 'it':
        # Translate Italian to English
        return italian_to_english(comment)[0]['translation_text']
    else:
        # No translation needed for English comments
        return comment

def translate_comment_by_seq(row):
    comment = row['Comments']
    lang = row['Language']
    
    # # Split the comment into sentences (adjust the splitting logic)
    # def split_sentences(text):
    #     sentences = []
    #     current_sentence = ""
    #     for token in text.split(". "):
    #         if token[0].isdigit() and len(token) > 1 and token[1] == ".":
    #             # Append the current sentence and start a new one
    #             sentences.append(current_sentence.strip())
    #             current_sentence = token
    #         else:
    #             current_sentence += ". " + token
    #     sentences.append(current_sentence.strip())  # Append the last sentence
    #     return sentences
    
    # sentences = split_sentences(comment)
    sentences = comment.split(". ")

    # Initialize an empty list to store translated sentences
    translated_sentences = []

    # Translate each sentence separately
    for sentence in sentences:
        if lang == 'fr':
            # Translate French to English
            try:
                translated_sentence = french_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        elif lang == 'de':
            # Translate German to English
            try:
                translated_sentence = german_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        elif lang == 'it':
            # Translate Italian to English
            try :
                translated_sentence = italian_to_english(sentence)[0]['translation_text']
            except IndexError:
                translated_sentence = sentence
        else:
            # No translation needed for English comments
            translated_sentence = sentence

        # Append the translated sentence to the list
        translated_sentences.append(translated_sentence)

    # Combine the translated sentences
    full_translation = ". ".join(translated_sentences)
    return full_translation

# Apply translation to each row
# df_merge['translated_comments'] = df_merge.apply(translate_comment, axis=1)
df_merge[3607:3610].apply(translate_comment_by_seq, axis=1)

3642    role model, clear communication, expertise, fa...
3643    The supervisor belongs to the team, clear goal...
3644    1. Trust in the MA; 2. Helpiveness; 3. Sparrin...
dtype: object

In [38]:
# 1130 et 1144 dans le csv 
text1 = """Ich erwarte von einem VG, dass er über objektiv messbare Kern-Handlungs-Kompetenzen verfügt. Diese sind meiner Meinung nach: Selbstwahrnehmung und Selbstreflexionsfähigkeit  >> Hier geht es vor allem darum, sowohl positive als auch negative Ursachen auf sich zuschreiben zu können; Selbstregulierung >> Damit ist in erster Linie die Impulskontrolle gemeint;  Empathie und Fähigkeit zum Perspektivenwechsel    >> Fähigkeit, seine eigene, innere,  individuelle, subjektive Wirklichkeit zu konstruieren und zu vertreten. Nur so kann ich andere auch wirklich verstehen, dem gesagten Sinn geben, ohne aber unbedingt damit einverstanden sein zu müssen. Hierhin gehört auch der abgedroschene Begriff "Vertrauen". Erst wenn ich jemanden richtig kenne kann ich ihm trauen, es entsteht Respekt und Vertrauen; Fähigkeit zur Integration     >> Damit ist in erster Linie gemeint, dass Unterschiede als Chance und nicht als Gefahr/Konkurrenz gesehen werden und dass der Fokus auf das Verbindende statt auf das Trennende gelegt wird. Ansonsten hat „Integrieren“ viel mit Empathie zu tun; Lern- und Veränderungsbereitschaft  >> Es geht darum, auf dem Hintergrund der Selbstreflexion und der Offenheit für Feedbacks von anderen das eigene Wissen und Können auch anpassen zu wollen bzw. zu können. Hier hin gehört auch die Fähigkeit, Kritik annehmen zu können, ohne verletzt und unkontrolliert darauf zu reagieren. Kommunikationsfähigkeit >> Insbesondere die Fähigkeit, auch heikle Botschaften an andere senden zu können. Visionäres, zielorientiertes Denken und Handeln  >>Viele Menschen erkennen, was nicht funktioniert, sind aber weniger gut in der Lage, nach dem Soll (wie müsste es denn anstelle von … sein?) zu fragen und entsprechende Massnahmen zur Soll-Erreichung zu generieren. Aber es ist noch viel mehr darin enthalten: Inspiration, Sinn stiften, motivieren, indem man selber Visionen hat und seine MA „mitziehen“ kann. Integrität im Handeln   >>Handlungen und Entscheidungen stimmen mit dem überein, was die FK sagt. Dadurch wird die FK berechenbar und verlässlich. Fähigkeit zur (Selbst-) Motivation    >>Leistungsdrang, Engagement, Initiative und Optimismus. Management-Fähigkeiten >>Niemand will einen Chaoten zum Chef, der sich und andere nicht organisieren kann. Stabilität ist ein wichtiger Sicherheitsfaktor."""

#2418 dans le csv, 2391 dans iloc
text2 = """Ein guter Vorgesetzter vertraut seinen MA, er zeigt auch seine Wertschätzung durch lobende Worte und indem Erfolge/überstandene Druckphase z.B. mit einem Apéro gefeiert werden. Ein guter Vorgesetzter fordert und fördert indem er seinen MA (wenn der MA sich bewiesen hat) indem er immer weitere Kompetenzen und Verantwortungen überträgt und dies auch mit einer Lohnerhöhung wertschätzt (v.a. MA die sehr zuverlässig/produktiv/initiativ/mitdenkend sind). Ein guter Vorgesetzter greift Ideen seiner MA auf und probiert diese aus (er ist nicht der Allwissende und zeigt das auch indem er proaktiv Input einholt und im Team die Kreativität fördert --> Voraussetzung dafür ist aber eine Vertrauensbasis). Ein guter Vorgesetzter hört seinen MA zu und platziert in einem Meeting nicht nur seine Botschaften sondern lässt Raum für kritische Stimmen/Befürwortung/eigene Anliegen und nimmt diese auch ernst (keine Alibiübungen!). Ein guter Vorgesetzter kennt die Tätigkeiten seiner MA, er hat ein gutes Timing in der Vergabe der Aufträge (wenn alle schon eh überlastet sind, unwichtiges/nicht dringendes auch noch einfordern, nur mit dem Ziel, dass er es vom Tisch hat). Ein guter Vorgesetzter weiss (oder hat ein Gespür), wenn er mehr verlangen (also Druck erhöhen kann) und wann nicht. Er kennt also die Auslastung seiner Mitarbeiter und auch deren aktuellen Belastbarkeit (z.B. können durch familiäre/verwandtschaftliche/Freunde/Bekannte Tragödien/Unfälle/Ausfälle/etc. die aktuelle Leistungen/Belastbarkeiten gemindert/eingeschränkt sein). Ein guter Vorgesetzter setzt sportliche Ziele, ist aber auch flexibel, wenn sich die Zielsetzung im laufe des Jahres verschiebt (z.B. neue Projekte die nicht in der Zielvereinbarung sind aber sehr viel Ressourcen beanspruchen, so dass die Ziele in der Zielvereinbarung nicht mehr erfüllt werden können, dafür hat man jedoch andere Ziele erreicht die wichtiger waren). Ein guter Vorgesetzter informiert seine MA früh in einem Änderungsprozess und lässt eine Diskussion zu (natürlich nicht ausufernd, aber er hört sich die Gründe an und berücksichtigt mögliche Einsprachen, wenn dies Sinn macht). Ein guter Vorgesetzter ist nicht fehlerfrei und gesteht das auch ein (er ist nicht der makellose Chef) und akzeptiert, dass dort wo gearbeitet wird auch Fehler entstehen, insbesondere wenn die Zeit sehr knapp und der Druck sehr hoch ist. Grundsätzlich sollte der Vorgesetze diese Leitsprüche leben: Ich Vertraue meinem Team / Mein Team ist meine Stütze / Ich wertschätze die Arbeite meiner MA und gebe Feedback (z.B. in einer Team Retrospektive die von jemand neutralem moderiert wird) / Ich kenne die Stärken und Schwächen meiner MA / Ich baue auf die Stärken meiner MA und setze diese optimal ein / Mein Team und ich kennen die Ziele und verfolgen diese gemeinsam und mit aller kraft / Mein Team und ich reagieren flexibel in gemeinsamem Austausch auf Veränderungen (von innen und aussen) / Mein Team und ich setzten Standards in der Unternehmung / Wir wollen ZUSAMMEN die Besten sein - alle ziehen an einem Strick und dafür schaffe ICH die Voraussetzungen durch meine Führungskompetenzen """

# 3453 iloc, 3490 dans le csv
text3 = """Le bon supérieur doit conduire son équipe vers l’objectif attendu, porter les valeurs de l’entreprise, donner des responsabilités et de l’autonomie, et contribuer à définir les méthodes, les priorités et les moyens. J’attends de lui de l’expérience, et du bon sens, tout en ayant l’humilité de savoir s’appuyer sur celle de son équipe. J’apprécie qu’il sache dénicher les meilleurs talents pour l’aider à compléter ses propres compétences et axes de progrès (que ce soit à l’interne et via recrutement externe si besoin), tout en faisant preuve de leadership avec des qualités humaines évidentes. J’ai besoin que mon supérieur sache prendre des décisions, sans pour autant refuser de se remettre en question. Également, il doit savoir féliciter, recadrer, rester juste, et s’intéresser à son équipe en proximité. Il me semble par ailleurs important qu’il sache gérer son stress et rester serein, savoir gérer les conflits, être motivant, patient lorsque c’est nécessaire, intéressé et disponible pour son équipe, sans pour autant infantiliser ou déresponsabiliser ses collaborateurs afin que chacun puisse garder son rôle et sa crédibilité au sein de l’organigramme. Il doit réussir à faire faire, sans faire à la place des autres ; il doit savoir demander conseil, et ne pas s’enfermer dans son rôle en cas de difficulté."""

# 3610 iloc, 3646 csv
text4 = """1. Vertrauen in die MA; 2. Hilfsbereitschaft; 3. Sparring-Partner zum Hinterfragen neuer Ideen; 4. Offenes Ohr; 5. Coaching"""


# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

# # Tokenize each comment and count tokens
# def count_tokens(comment):
#     tokens = tokenizer.tokenize(comment)
#     return len(tokens)

# # Apply the function to each row
# df_merge['token_count'] = df_merge['Comments'].apply(count_tokens)

# # Find the maximum token count
# max_token_count = df_merge['token_count'].max()

# print(f"Max token count in the DataFrame: {max_token_count}")

/home/cattiaux/anaconda3/envs/wassati/lib/python3.9/site-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


Max token count in the DataFrame: 1346


In [42]:
german_to_english("""1""")[0]['translation_text']

# Split the text into sentences (you can adjust the splitting logic)
# sentences = text4.split(". ")

# # Translate each sentence separately
# translated_sentences = []
# for sentence in sentences:
#     translated_sentence = german_to_english(sentence)[0]['translation_text']
#     translated_sentences.append(translated_sentence)

# # Combine the translated sentences
# full_translation = ". ".join(translated_sentences)
# full_translation

IndexError: index out of range in self

In [34]:
df_merge.iloc[3609:3610]

,ID,Startzeit,Fertigstellungszeit,E-Mail,Name,Language,Comments,is_boss,coach,comment_to_translate,token_count
3644,3645.0,2021-06-21 17:15:52,2021-06-21 17:18:55,anonymous,NaN,de,1. Vertrauen in die MA; 2. Hilfsbereitschaft; ...,NaN,NaN,Translate German to English: 1. Vertrauen in d...,49


In [42]:
def isValid_cpf(cpf):
    # Remove any formatting
    cpf = ''.join(filter(str.isdigit, cpf))
    
    # Check if CPF has 11 digits
    if len(cpf) != 11:
        return False
    
    # Check if CPF has all digits equal
    if cpf == cpf[0] * len(cpf):
        return False
    
    # Calculate first check digit
    first_check_sum = sum(int(digit) * weight for digit, weight in zip(cpf[:9], range(10, 1, -1)))
    first_check_digit = (first_check_sum * 10 % 11) % 10
    if int(cpf[9]) != first_check_digit:
        return False
    
    # Calculate second check digit
    second_check_sum = sum(int(digit) * weight for digit, weight in zip(cpf[:10], range(11, 1, -1)))
    second_check_digit = (second_check_sum * 10 % 11) % 10
    if int(cpf[10]) != second_check_digit:
        return False
    
    return True

isValid_cpf("11418074403")

def complete_cpf(incomplete_cpf, nb_possibilities=None):
    def calculate_check_digit(digits, position):
        sum_of_products = sum(int(digit) * weight for digit, weight in zip(digits, range(position, 1, -1)))
        check_digit = (sum_of_products * 10 % 11) % 10
        return str(check_digit)

    def generate_possibilities(cpf, missing_indices, current_index, possibilities):
        if current_index == len(missing_indices):
            # Only calculate check digits if they are missing (marked by asterisks)
            if cpf[9] == '*':
                cpf[9] = calculate_check_digit(cpf[:9], 10)
            if cpf[10] == '*':
                cpf[10] = calculate_check_digit(cpf[:10], 11)
            completed_cpf_str = ''.join(cpf)
            if isValid_cpf(completed_cpf_str) and completed_cpf_str not in possibilities:
                possibilities.append(completed_cpf_str)
            return

        for digit in '0123456789':
            cpf[missing_indices[current_index]] = digit
            generate_possibilities(cpf, missing_indices, current_index + 1, possibilities)

    # Remove formatting and find missing indices
    cpf = list(incomplete_cpf.replace('.', '').replace('-', ''))

    missing_indices = [i for i, digit in enumerate(cpf) if digit == '*']

    if len(cpf) != 11 or len(missing_indices) == 0:
        return []

    possibilities = []
    generate_possibilities(cpf, missing_indices, 0, possibilities)

    if nb_possibilities is not None:
        return possibilities[:nb_possibilities]
    return possibilities

complete_cpf('063.387.*47-0*')  # Returns all valid CPF completions

['06338734703', '06338764700']